In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df = pd.read_csv('mw_pw_profiles (1).csv')

<ipython-input-6-55241279>:1: DtypeWarning: Columns (1,32,45) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('mw_pw_profiles (1).csv')


In [ ]:
def get_dataframe_info(df):

    info_dict = {
        'Column_Name': df.columns,
        'Data_Type': df.dtypes,
        'NaN_Count': df.isna().sum(),
        'NaN_Percentage': (df.isna().sum() / len(df) * 100).round(2),
        'Description': [f"Column '{col}' is of type {df[col].dtype} with {df[col].isna().sum()} missing values."
                       for col in df.columns]
    }

    print(pd.DataFrame(info_dict))
get_dataframe_info(df)

                                 Column_Name Data_Type  NaN_Count  \
player_id                          player_id    object          0   
match_id                            match_id    object          0   
gender                                gender    object          0   
balls_per_over                balls_per_over     int64          0   
start_date                        start_date    object          0   
series_name                      series_name    object        408   
match_type                        match_type    object          1   
name_x                                name_x    object          1   
runs_scored                      runs_scored   float64          1   
player_out                        player_out   float64          1   
balls_faced                      balls_faced   float64          1   
fours_scored                    fours_scored   float64          1   
sixes_scored                    sixes_scored   float64          1   
catches_taken                  cat

In [ ]:
df = df[['player_id','start_date', 'catches_taken', 'run_out_direct', 'run_out_throw', 'stumpings_done', 'balls_bowled',
              'runs_conceded', 'wickets_taken', 'maidens',
              'runs_scored', 'balls_faced', 'fours_scored', 'sixes_scored', 'fantasy_score_total']]
df

,player_id,start_date,catches_taken,run_out_direct,run_out_throw,stumpings_done,balls_bowled,runs_conceded,wickets_taken,maidens,runs_scored,balls_faced,fours_scored,sixes_scored,fantasy_score_total
0,15688,21-08-18,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.0,30.0,1.0,0.0,17.0
1,15688,05-05-19,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0,18.0,1.0,0.0,7.0
2,15688,09-09-21,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,5.0,0.0,0.0,5.0
3,15688,15-09-21,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0
4,15688,14-09-21,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,15.0,0.0,0.0,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90034,399b0b94,03-06-21,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,64.0,103.0,10.0,1.0,88.0
90035,399b0b94,04-07-21,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,34.0,103.0,4.0,0.0,46.0
90036,399b0b94,11-07-21,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,22.0,51.0,3.0,0.0,33.0
90037,399b0b94,06-03-21,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,21.0,84.0,2.0,0.0,31.0


In [ ]:
def calculate_ema_stats(df, span=14):
    df = df.copy()
    df['start_date'] = pd.to_datetime(df['start_date'])
    df = df.sort_values(['player_id', 'start_date'])


    stat_columns = [col for col in df.columns if col not in ['player_id', 'start_date']]

    result = df[['player_id']].copy()

    for col in stat_columns:
        result[f'{col}_ema'] = df.groupby('player_id')[col].transform(
            lambda x: x.ewm(span=span, adjust=False).mean()
        )

    result = result.loc[df.groupby('player_id')['start_date'].idxmax()]

    final_columns = ['player_id'] + [col for col in result.columns if col.endswith('_ema')]
    result = result[final_columns]

    return result
df = calculate_ema_stats(df)
df

<ipython-input-9-2099705641>:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['start_date'] = pd.to_datetime(df['start_date'])


,player_id,catches_taken_ema,run_out_direct_ema,run_out_throw_ema,stumpings_done_ema,balls_bowled_ema,runs_conceded_ema,wickets_taken_ema,maidens_ema,runs_scored_ema,balls_faced_ema,fours_scored_ema,sixes_scored_ema,fantasy_score_total_ema
11,00029c30,0.000000,0.0000,0.000000,0.000000,11.726242,14.373774,0.730086,0.000000,0.491496,0.997823,0.000000,0.000000,24.349301
12,0019ca13,0.000000,0.0000,0.000000,0.000000,6.000000,8.000000,0.000000,0.000000,1.000000,2.000000,0.000000,0.000000,1.000000
32,0030a57d,0.568483,0.0000,0.068655,0.329083,0.000000,0.000000,0.000000,0.000000,0.692458,2.226110,0.000000,0.000000,12.529592
64,00321fff,0.076681,0.0000,0.000000,0.000000,16.239733,18.829289,0.720689,0.029666,1.340845,1.937817,0.121957,0.000000,28.588857
132,003d49e6,0.423060,0.0000,0.002425,0.000000,11.342682,15.596999,0.205820,0.200425,8.512050,6.577126,0.701626,0.387214,23.006045
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9844,7160156,0.100148,0.0000,0.000000,0.000000,1.340890,2.117115,0.231111,0.000000,1.364168,6.451844,0.133333,0.000000,12.467575
379,784913,0.454047,0.0565,0.000000,0.000000,35.231559,28.918509,1.394232,0.056500,2.392818,4.286695,0.265489,0.065193,46.324317
10830,8320616,0.000000,0.0000,0.000000,0.000000,17.493333,19.226074,0.115556,0.000000,1.466667,5.552889,0.000000,0.000000,6.496296
666,863408,0.965501,0.0000,0.048967,0.098938,10.352584,11.283026,0.532915,0.000000,17.507441,22.380575,1.577763,0.577250,49.368981


In [ ]:
df.to_csv('/data.csv')

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import RFE
from sklearn.metrics import mean_squared_error, r2_score
import xgboost as xgb
import shap

In [ ]:
def train_random_forest(df, target_col, feature_cols=None, n_estimators=100, random_state=42):
    if feature_cols is None:
        feature_cols = [col for col in df.columns if col not in ['player_id', target_col]]

    X = df[feature_cols]
    y = df[target_col]

    model = RandomForestRegressor(n_estimators=n_estimators, random_state=random_state)
    model.fit(X, y)

    predictions = model.predict(X)

    mse = mean_squared_error(y, predictions)
    r2 = r2_score(y, predictions)

    print("Random Forest Regressor Results:")
    print(f"Mean Squared Error: {mse:.4f}")
    print(f"R² Score: {r2:.4f}")

    importance = pd.DataFrame({
        'Feature': feature_cols,
        'Importance': model.feature_importances_
    }).sort_values(by='Importance', ascending=False)
    print("\nFeature Importance:")
    print(importance)

    output_df = df[['player_id',  target_col]].copy()
    output_df['predicted_value'] = predictions

    return model, output_df, importance



In [ ]:
def train_xgboost(df, target_col, feature_cols=None, n_estimators=100, random_state=42):
    if feature_cols is None:
        feature_cols = [col for col in df.columns if col not in ['player_id', target_col]]

    X = df[feature_cols]
    y = df[target_col]

    model = xgb.XGBRegressor(n_estimators=n_estimators, random_state=random_state)
    model.fit(X, y)

    predictions = model.predict(X)

    mse = mean_squared_error(y, predictions)
    r2 = r2_score(y, predictions)

    print("XGBoost Regressor Results:")
    print(f"Mean Squared Error: {mse:.4f}")
    print(f"R² Score: {r2:.4f}")

    importance = pd.DataFrame({
        'Feature': feature_cols,
        'Importance': model.feature_importances_
    }).sort_values(by='Importance', ascending=False)
    print("\nFeature Importance:")
    print(importance)

    output_df = df[['player_id',  target_col]].copy()
    output_df['predicted_value'] = predictions

    return model, output_df, importance


In [ ]:
def apply_pca(df, feature_cols=None, n_components=2):
    if feature_cols is None:
        feature_cols = [col for col in df.columns if col not in ['player_id', ]]

    X = df[feature_cols]

    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    pca = PCA(n_components=n_components)
    X_pca = pca.fit_transform(X_scaled)

    print("PCA Results:")
    print(f"Explained Variance Ratio: {pca.explained_variance_ratio_}")
    print(f"Total Explained Variance: {np.sum(pca.explained_variance_ratio_):.4f}")

    pca_df = pd.DataFrame(X_pca, columns=[f'PC{i+1}' for i in range(n_components)])
    pca_df['player_id'] = df['player_id'].values


    return pca_df, pca, pca.explained_variance_ratio_



def calculate_shap_scores(model, df, feature_cols=None):
    if feature_cols is None:
        feature_cols = [col for col in df.columns if col not in ['player_id', ]]

    X = df[feature_cols]

    explainer = shap.TreeExplainer(model)
    shap_values = explainer.shap_values(X)

    shap_df = pd.DataFrame(shap_values, columns=[f'{col}_shap' for col in feature_cols])
    shap_df['player_id'] = df['player_id'].values

    print("SHAP Values Summary:")
    print(shap_df.describe())

    return shap_df

def apply_rfe(df, target_col, estimator, n_features_to_select=5, feature_cols=None):
    if feature_cols is None:
        feature_cols = [col for col in df.columns if col not in ['player_id',  target_col]]

    X = df[feature_cols]
    y = df[target_col]

    rfe = RFE(estimator=estimator, n_features_to_select=n_features_to_select)
    rfe.fit(X, y)

    selected_features = [feature_cols[i] for i in range(len(feature_cols)) if rfe.support_[i]]

    print("RFE Results:")
    print(f"Selected Features ({n_features_to_select}): {selected_features}")
    print(f"Feature Ranking: {rfe.ranking_}")

    return selected_features, rfe

In [ ]:
    df = pd.read_csv("/data.csv")
    rf_model, rf_output_df, rf_importance = train_random_forest(df, target_col='fantasy_score_total_ema')
    print("\nRandom Forest Output DataFrame:\n", rf_output_df.head())

    xgb_model, xgb_output_df, xgb_importance = train_xgboost(df.head(1000), target_col='fantasy_score_total_ema')
    print("\nXGBoost Output DataFrame:\n", xgb_output_df.head())

    pca_df, pca, variance_ratio = apply_pca(df)
    print("\nPCA Transformed DataFrame:\n", pca_df.head())



    shap_df = calculate_shap_scores(rf_model, df)
    print("\nSHAP Values DataFrame:\n", shap_df.head())

    estimator = RandomForestRegressor(random_state=42)
    selected_features, rfe = apply_rfe(df, target_col='fantasy_score_total_ema', estimator=estimator)
    print("\nSelected Features by RFE:", selected_features)

Random Forest Regressor Results:
Mean Squared Error: 2.8713
R² Score: 0.9941

Feature Importance:
               Feature  Importance
7    wickets_taken_ema    0.559032
9      runs_scored_ema    0.369282
1    catches_taken_ema    0.019693
6    runs_conceded_ema    0.010885
11    fours_scored_ema    0.010460
5     balls_bowled_ema    0.005026
10     balls_faced_ema    0.005011
12    sixes_scored_ema    0.004553
0           Unnamed: 0    0.004052
8          maidens_ema    0.003695
2   run_out_direct_ema    0.003461
3    run_out_throw_ema    0.002896
4   stumpings_done_ema    0.001953

Random Forest Output DataFrame:
   player_id  fantasy_score_total_ema  predicted_value
0  00029c30                24.349301        25.652948
1  0019ca13                 1.000000         1.936885
2  0030a57d                12.529592        12.889129
3  00321fff                28.588857        28.433435
4  003d49e6                23.006045        23.932560
XGBoost Regressor Results:
Mean Squared Error: 0.0538
